In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)
    
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, "route")
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name
    
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)
        
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
            
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        
        except Exception as e:
            print(f"Error occured while scraping bus details for {url}: {str(e)}")
            return []
        
    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []
    
all_bus_details = []

def scrape_all_pages():
    for page in range(1,5):
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(2)
                
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv("west_bengal_bus_details.csv", index = False)

driver.quit()
            
            

Error occured while scraping bus details for https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF63766FDA5+29557]
	(No symbol) [0x00007FF6375E2240]
	(No symbol) [0x00007FF63749B6EA]
	(No symbol) [0x00007FF6374EFA15]
	(No symbol) [0x00007FF6374EFC6C]
	(No symbol) [0x00007FF63753BB07]
	(No symbol) [0x00007FF63751753F]
	(No symbol) [0x00007FF6375388A3]
	(No symbol) [0x00007FF6375172A3]
	(No symbol) [0x00007FF6374E12DF]
	(No symbol) [0x00007FF6374E2451]
	GetHandleVerifier [0x00007FF63799DCBD+3363469]
	GetHandleVerifier [0x00007FF6379E9B47+3674391]
	GetHandleVerifier [0x00007FF6379DEAEB+3629243]
	GetHandleVerifier [0x00007FF63772FC66+815670]
	(No symbol) [0x00007FF6375ED6EF]
	(No symbol) [0x00007FF6375E92B4]
	(No symbol) [0x00007FF6375E9450]
	(No symbol) [0x00007FF6375D81FF]
	BaseThreadInitThunk [0x00007FFBC57C257D+29]
	RtlUserThreadStart [0x00007FFBC702AF28+40]



NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,05h 50m,09:50,2.1,INR 165,48 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,04h 30m,10:10,3.2,INR 161,40 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,03h 25m,12:25,2.7,INR 161,45 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,WBTC (CTC) HABRA-DIGHA via Bally - 26|10:00,Non AC Seater (2+3),10:00,04h 45m,14:45,3.4,INR 161,46 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,"WBTC (CTC) BARASAT-DIGHA via Esplanade, Karuna...",Non AC Seater (2+3),10:15,05h 20m,15:35,3.0,INR 165,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
615,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:55,Non AC Seater (2+3),06:55,03h 50m,10:45,4.2,INR 130,51 Seats available
616,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,03h 55m,16:40,4.2,INR 130,43 Seats available
617,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|14:45,Non AC Seater (2+3),14:45,02h 55m,17:40,4.0,INR 111,47 Seats available
618,Midnapore to Kolkata Airport,https://www.redbus.in/bus-tickets/midnapore-to...,WBTC (CTC) Habra-Midnapur - 194|15:40,Non AC Seater (2+3),15:40,03h 00m,18:40,3.8,INR 111,34 Seats available
